<a href="https://colab.research.google.com/github/ma-zeller/IANNwTF/blob/main/IANNwTF_HW_10_Group_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install -q tensorflow-text==2.9.0

import tensorflow as tf
import tensorflow_datasets as tfds   
from copy import deepcopy
from matplotlib import pyplot as plt
import numpy as np
from numpy.linalg import norm
import math
import datetime
from tqdm import tqdm
import regex as re
import tensorflow_text as tf_text

%load_ext tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 6.0 MB/s eta 0:00:00


In [2]:
# mount drive
import os
from google.colab import drive
drive.mount ("/content/drive")
os.chdir ("drive/MyDrive")

Mounted at /content/drive


In [3]:
# import and open file
corpus = open("/content/drive/MyDrive/bible.txt")

# convert object to string
corpus = corpus.read()
# print(corpus[0:250])

In [4]:
# hyperparameters
context_window = 4
vocab_size = 10000
batch_size = 64
embed_size = 64
learning_rate = 0.01
epochs = 10

In [5]:
# remove special chars
corpus = re.sub('[^A-Za-z\s.]+', '', corpus)

# remove newline
corpus = corpus.replace('\n', ' ').replace('\r', '')

# convert to lowercase
corpus = corpus.lower()

# #print result
# print(corpus[0:244])

In [6]:
# tokenize
corpus = tf_text.WhitespaceTokenizer().split(corpus)
print(f"Length: {len(corpus)}")

Length: 790017


In [7]:
# keep top k most common words
y, idx, count = tf.unique_with_counts(corpus)
filter = y[0:vocab_size]
corpus = corpus.numpy().tolist()
filter = filter.numpy().tolist()

In [8]:
# remove uncommen words from dataset (disgustingly inefficient, should use np mask and where)
for i, x in enumerate(tqdm(corpus)):
  if x in filter:
    continue
  else:
    corpus.remove(x)
print("Done")

 98%|█████████▊| 770324/790017 [03:05<00:04, 4156.34it/s]

Done


In [9]:
# back to list and reformat
corpus = " ".join(str(x).replace("b'", "").replace("'","") for x in corpus)

In [10]:
# tokenize again
corpus = tf_text.WhitespaceTokenizer().split(corpus)

In [11]:
print(f"New length: {len(corpus)}")
for i, word in enumerate(corpus):
  print(word)

  if i == 10:
    break

New length: 770324
tf.Tensor(b'the', shape=(), dtype=string)
tf.Tensor(b'first', shape=(), dtype=string)
tf.Tensor(b'book', shape=(), dtype=string)
tf.Tensor(b'of', shape=(), dtype=string)
tf.Tensor(b'moses', shape=(), dtype=string)
tf.Tensor(b'called', shape=(), dtype=string)
tf.Tensor(b'genesis', shape=(), dtype=string)
tf.Tensor(b'in', shape=(), dtype=string)
tf.Tensor(b'the', shape=(), dtype=string)
tf.Tensor(b'beginning', shape=(), dtype=string)
tf.Tensor(b'god', shape=(), dtype=string)


In [12]:
# Get targets by creating sequences from corpus
targets = list()
for i, elem in tqdm(enumerate(corpus)):
  
  if i == (len(corpus)-1-context_window):
    break

  else:
    targets.append(corpus[i+1:i+1+context_window])

770319it [04:39, 2752.07it/s]


In [13]:
print(f"Length before cutoff: {len(corpus)}")
print(f"Target length: {len(targets)}")

# adjust length
corpus = corpus[:len(corpus)-(len(corpus)-len(targets))]
print(f"Length after cutoff: {len(corpus)}")


Length before cutoff: 770324
Target length: 770319
Length after cutoff: 770319


In [14]:
corpus_ds = tf.data.Dataset.from_tensor_slices(corpus)
target_ds = tf.data.Dataset.from_tensor_slices(targets)

In [15]:
ds = tf.data.Dataset.zip((corpus_ds, target_ds))

In [16]:
def ds_preprocess(bible):

    # batch
    bible = bible.batch(batch_size)

    # shuffle
    bible = bible.shuffle(10000)

    # prefetch
    bible = bible.prefetch(tf.data.AUTOTUNE)
    
    return bible

In [17]:
bible_ds = ds.apply(ds_preprocess)

print(f"Number training batches: {len(bible_ds)}")

Number training batches: 12037


In [18]:
for text, target in bible_ds:
  print(f"Text: {text[0]}")
  print(f"Target: {target[0]}")
  break

Text: b'up.'
Target: [b'the' b'princes' b'refrained' b'talking']


In [19]:
# https://github.com/svaderia/SkipGram/blob/master/skipgram.py

In [25]:
class SkipGramModel(tf.keras.layers.Layer):

    def __init__(self, vocab_size, embed_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size

        self.lr = learning_rate
        self.optimizer = tf.keras.optimizers.Adam(1e-4)
        
  
    def build(self):
        self.embedding_matrix = self.add_weight(shape=(self.vocab_size, self.embed_size),initializer="uniform",trainable=True)
        self.score_matrix = self.add_weight(shape=(self.vocab_size, self.embed_size),initializer="uniform",trainable=True)

    def call(self,inputs):
        embed = tf.nn.embedding_lookup(self.embedding_matrix, np.asarray(self.score_matrix,dtype=np.int32))

        nce_weight = tf.Variable(tf.truncated_normal([self.vocab_size, self.embed_size],
                                                        stddev=1.0 / (self.embed_size ** 0.5)), 
                                                        name='nce_weight')
        
        nce_bias = tf.Variable(tf.zeros([self.vocab_size]), name='nce_bias') 
        self.loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weight, 
                                                        biases=nce_bias, 
                                                        labels=inputs, 
                                                        inputs=embed, 
                                                        num_sampled=self.num_sampled, 
                                                        num_classes=self.vocab_size), name='loss')  

    def train_step(self,data):
      with tf.GradientTape() as tape:

            # define a pair of a word and a target
            input = data[0]
            context = data[1]

            input_embedding = self.call(input)

            context_embedding = self.call(context)
 
            loss = self.loss_function(context_embedding,input_embedding)

      gradients = tape.gradient(loss, self.trainable_variables)
      self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

      return loss

In [21]:
# instantiate model
model = SkipGramModel(vocab_size, embed_size)

In [23]:
# summary
model.build()

In [27]:
# Show result

fig = plt.figure(figsize=(8, 8))
columns = 4
rows = 2

# initiate a loss list
loss_lst = []

def train(model, data):
    # train the embedding model
    for epoch in range(epochs):
        print(f"Epoch: {epoch}")

        for pair in tqdm(data):
            # pair is one two-word list from the sequences list of lists (the input/context pairs)

            loss = model.train_step(pair)
            loss_lst.append(loss)

        # after training, the weight matrix of the model's embedding layer is the embedding space lookup_table
        # access this so yu can work with the ultimate embedding space
        embedding_matrix = model.embedding.weights[0]

    return loss_lst, embedding_matrix

def test_embeddings(test_index, embedding_list, word_index):

    # embedding of text index
    test_embed = embedding_list[test_index]

    # A list for the cosine similarities
    cosine_sim = []

    # for each embedding in the embedding space
    for embed in embedding_list:

        # exclude the embedding for the test word itself
        if np.array_equal(embed,test_embed) != True:

            # compute the cosine similarity between the test word embedding and the current comparison embedding
            cosine = np.dot(test_embed, embed) / (norm(test_embed) * norm(embed))

            # add that cosine similarity value to the above-created list
            cosine_sim.append(cosine)

def loss_plot(loss_lst):
    fig = plt.figure()
    line1, = plt.plot(loss_list)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend([line1], ["Loss"])
    plt.title(f"Loss with {epochs} epochs")
    plt.show()

# train the model and acquire the embeddings and the loss list for plotting
loss_list, embeddings = train(model, bible_ds)

# plot the loss
loss_plot(loss_list)

Epoch: 0


  0%|          | 0/12037 [00:01<?, ?it/s]


InvalidArgumentError: ignored

<Figure size 576x576 with 0 Axes>

In [ ]:
EXPERIMENT_NAME = "IANN_wTF_HW_10"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logging_callback = tf.keras.callbacks.TensorBoard(log_dir=f"./logs/{EXPERIMENT_NAME}/{current_time}")

In [ ]:
#!kill 11736
%tensorboard --logdir logs/

##To DO:

Kann ich die zweite variante auch benutzen?
Transfer learning?
Loss function